<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/main/20221025_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
from datetime import datetime, timedelta


from tqdm.autonotebook import tqdm
import csv

import re

import gzip
import shutil

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


In [2]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Mounted at /content/gdrive/


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [3]:
admission = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv', low_memory=False, dtype='str', keep_default_na=False)
patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv', low_memory=False, dtype='str', keep_default_na=False)
diagnoses_icd = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/diagnoses_icd.csv', low_memory=False, dtype='str', keep_default_na=False)
icustays = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/icu/icustays.csv', low_memory=False, dtype='str', keep_default_na=False)

In [4]:
sepsis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv', low_memory=False, dtype='str', keep_default_na=False)

In [5]:
admission = pd.merge(left = admission, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
patients = pd.merge(left = patients, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
diagnoses_icd = pd.merge(left = diagnoses_icd, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
sepsis = pd.merge(left = sepsis, right = icustays[['hadm_id','stay_id']], how='left', left_on='stay_id', right_on='stay_id')

In [6]:
Total_HADMIDS = set(sepsis['hadm_id'])

In [7]:
EXTRACT_FEATURE_CODES_VS = dict({
    '220045': 'PULSE',
    '211': 'PULSE',
    '618': 'RESP',
    '220210': 'RESP',
    '220050': 'SBP',
    '220179': 'SBP',
    '225309': 'SBP',
    '51': 'SBP',
    '455': 'SBP',
    '220051': 'DBP',
    '220180': 'DBP',
    '225310': 'DBP',
    '8368': 'DBP',
    '8441': 'DBP',
    '223762': 'TEMP',
    '676': 'TEMP',
    '677': 'TEMP',
    '223761': 'TEMP_F',
    '678': 'TEMP_F',
    '679': 'TEMP_F',
    '223835' : 'FiO2',
    '189' : 'FiO2',
    '190' : 'FiO2',
    '220052' : 'MAP',
    '1321' : 'MAP',
    '438' : 'MAP'
})

EXTRACT_FEATURE_CODES_LAB = dict({
    '50885': 'BILIRUBIN',
    '220546': 'WBC',
    '51301': 'WBC',
    '51221': 'HEMATOCRIT',
    '225668': 'LACTATE',
    '50813': 'LACTATE',
    '51265': 'PLATELET',
    '50912': 'CREATININE',
    '220615': 'CREATININE',
    '229761': 'CREATININE',
    '50983': 'SODIUM',
    '50820': 'PH',
    '50831': 'PH',
    '50971': 'POTASSIUM'
})

FEATURE_NORMAL_RANGE = dict({
    'PULSE': (0, 300),
    'SBP': (0, 300),
    'DBP': (0, 300),
    'RESP': (0, 120),
    'TEMP': (25, 50),
    'BILIRUBIN': (0, 75),
    'WBC': (0, 100),
    'HEMATOCRIT': (0, 100),
    'LACTATE': (0, 20),
    'PLATELET': (0, 1000),
    'CREATININE': (0, 20),
    'SODIUM': (0, 500) 
})

# VITAL_SIGNS = {'PULSE': ['220045', '211'],
#                'RESP': ['220210', '618'],
#                'SBP': ['220050', '220179', '225309', '51', '455'],
#                'DBP': ['220051', '220180', '225310', '8368', '8441'],
#                'FiO2': ['223835', '189', '190'],
#                'MAP': ['220052', '1321', '438']
#                }
# LABS = {'BILIRUBIN': ['50885'],
#         'LACTATE': ['50813'],
#         'pH': ['50820', '50831'],
#         'SODIUM': ['50983'],
#         'POTASSIUM': ['50971'],
#         'CREATININE': ['50912'],
#         'WBC': ['51301'],
#         'HEMATOCRIT': ['51221']
#         }

# VITAL_SIGNS = {'PULSE': ['220045', '211'],
#                'RESP': ['220210', '618'],
#                'SBP': ['228152','229899','220050', '220179', '225309', '51', '455','224167','227243'],
#                'DBP': ['220051', '220180', '225310', '8368', '8441'],
#                'FiO2': ['229280','229841','226754','227009','227010', '223835', '189', '190'],
#                'MAP': ['220052', '1321', '438']
#                }

# LABS = {'lactate': ['225668'],
#         'glucose'	: ['220621','225664','226537','228388'],
#         'creatinine' : ['220615','229761'],
#         'WBC'	: ['220546','51301'],
#         'heart rate' : ['220046','220047','226764','220045','227018'],
#         'sodium' : ['220645','226534','228389','228390'],
#         'temperature' : ['223762','226329','227632','227634'],
#         'platelet' : ['227457'],
#         'hemoglobin' : ['220228'],
#         'pao2' : ['220224','227516','227039'],
# }

In [8]:
## Define some util functions
NOT_CONVERTED = 'NOT_CONVERTED'
## 문자로 기록된 날짜 값들을 파이썬의 datetime 데이터로 바꿔주는 함수 작성
def str2datetime(s):
    def _convert(_s, _dformat):
        try:
            converted_dt = datetime.strptime(_s, _dformat)
        except Exception:
            return NOT_CONVERTED

        return converted_dt

    if isinstance(s, datetime):
        return s

    dformats = [
        '%Y-%m-%d %p %I:%M:%S',
        '%Y-%m-%d %H:%M:%S.%f',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d %H:%M',
        '%y-%m-%d %H:%M',
        '%Y-%m-%d',
        '%Y%m%d',
        '%Y-%m-%d +%H:%M',
        '%Y%m%d %H%M',
        '%Y%m%d%H%M%S',
        ]

    s = s.split('.')[0]
    s = s.replace('오전', 'am').replace('오후', 'pm')
    for dformat in dformats:
        dt = _convert(s, dformat)
        if dt != NOT_CONVERTED:
            return dt

    return NOT_CONVERTED

In [9]:
def extract_features():
  features_hadm_id = dict()
  chartevents = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_unzip/icu/chartevents.csv', 'r', encoding='utf-8'))
  labevents = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_unzip/hosp/labevents.csv', 'r', encoding='utf-8'))
  for row in tqdm(chartevents, desc='Extract_VS', total=7241351, ncols=700):
    if row['hadm_id'] in Total_HADMIDS:
      hadm_id = row['hadm_id']
      features_hadm_id[hadm_id] = features_hadm_id.get(hadm_id, dict()) #get 없으면 2번째로 return

      item_id = row['itemid']
      if item_id not in EXTRACT_FEATURE_CODES_VS:
        continue
      item_name = EXTRACT_FEATURE_CODES_VS[item_id]
      try:
        #정규식을 이용해서 기록된 값에서 숫자 형태의 값만 가져오기
        #소수점 둘째자리 까지만 값을 사용하기
        item_value = round(float(re.findall("\d+\.\d+|\d+", row['valuenum'])[0]), 2)
      except Exception:
        continue
      
      item_time = str2datetime(row['charttime'])
      if item_time == NOT_CONVERTED:
        continue
      #GCP total score도 시간대에 총합으로 사용함
      if item_name == 'TEMP_F':
        item_value = round((item_value - 32.0) / 1.8, 2)
        item_name = 'TEMP'

    #   if item_value > FEATURE_NORMAL_RANGE[item_name][1]:
    #     item_value = FEATURE_NORMAL_RANGE[item_name][1]
    #   elif item_value < FEATURE_NORMAL_RANGE[item_name][0]:
    #     item_value = FEATURE_NORMAL_RANGE[item_name][0]

      features_hadm_id[hadm_id][item_name] = features_hadm_id[hadm_id].get(item_name, list()) #같은 환자의 여러개 feature가 있으면 list로 반환하도록
      features_hadm_id[hadm_id][item_name].append((item_time, item_value))
      
    else:
      continue

  for row in tqdm(labevents, desc='Extract_LAB', total=615328, ncols=700):
    if row['hadm_id'] in Total_HADMIDS:
      hadm_id = row['hadm_id']
      features_hadm_id[hadm_id] = features_hadm_id.get(hadm_id, dict())

      item_id = row['itemid']
      if item_id not in EXTRACT_FEATURE_CODES_LAB:
        continue
      item_name = EXTRACT_FEATURE_CODES_LAB[item_id]
      try:
        #정규식을 이용해서 기록된 값에서 숫자 형태의 값만 가져오기
        #소수점 둘째자리 까지만 값을 사용하기
        item_value = round(float(re.findall("\d+\.\d+|\d+", row['valuenum'])[0]), 2)
      except Exception:
        continue
      
      item_time = str2datetime(row['charttime'])
      if item_time == NOT_CONVERTED:
        continue

      #if item_value > FEATURE_NORMAL_RANGE[item_name][1]:
      #  item_value = FEATURE_NORMAL_RANGE[item_name][1]
      #elif item_value < FEATURE_NORMAL_RANGE[item_name][0]:
      #  item_value = FEATURE_NORMAL_RANGE[item_name][0]
        
      features_hadm_id[hadm_id][item_name] = features_hadm_id[hadm_id].get(item_name, list())
      features_hadm_id[hadm_id][item_name].append((item_time, item_value))
      
    else:
      continue
      
  return features_hadm_id

In [10]:
extracted_features = extract_features()

Extract_VS:   0%|                                                                                             …

Extract_LAB:   0%|                                                                                            …

In [11]:
# import pickle
# import gzip
# with gzip.open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_extracted_features_raw.pickle', 'wb') as f:
#     pickle.dump(extracted_features, f)

In [12]:
# from itertools import islice
# dict(list(extracted_features.items())[0:2])

In [13]:
# 사망 레이블 하기
death_labels = dict()
admissions = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_admissions.csv', 'r', encoding='utf-8'))
for row in tqdm(admissions, desc='Get Death Label'):
  hadm_id = row['hadm_id']
  death_yn = int(row['hospital_expire_flag'])
  if death_yn:
    event_time = str2datetime(row['deathtime'])
  else:
    event_time = str2datetime(row['dischtime'])
  death_labels[hadm_id] = dict({'DEATH': death_yn, 'EVENT_TIME': event_time})

Get Death Label: 0it [00:00, ?it/s]

In [14]:
# 재실 기간 레이블 하기
los_labels = dict()
admissions = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_admissions.csv', 'r', encoding='utf-8'))
for row in tqdm(admissions, desc='Get loss Label'):
  hadm_id = row['hadm_id']
  admin_time = str2datetime(row['admittime'])
  disc_time = str2datetime(row['dischtime'])
  los = (disc_time - admin_time).total_seconds() / (60*60)
  los_labels[hadm_id] = int(los)

Get loss Label: 0it [00:00, ?it/s]

In [15]:
## 환자의 입실 퇴실 시간 정보 가져오기
Extracted_HADMIDS = list(extracted_features.keys())
admin_disc_time = dict()

admissions = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_admissions.csv', 'r', encoding='utf-8'))
for row in admissions:
  hadm_id = row['hadm_id']
  admin_time = str2datetime(row['admittime'])
  disc_time = str2datetime(row['dischtime'])

  if (admin_time != NOT_CONVERTED) and (disc_time != NOT_CONVERTED):
    admin_disc_time[hadm_id] = dict({'ADMIN': admin_time, 'DISC': disc_time})

In [16]:
## 입실 시점으로부터 1시간 간격으로 데이터를 읽어오는 함수 정의
def quantize_features(feature_list, admin_time, disc_time, missing_feature=False):
  total_hours = int((disc_time - admin_time).total_seconds() / (60*60))
  #입실 이후 최대 24시간 동안의 데이터만 가져오기
  if total_hours > 24:
    total_hours = 24
  
  quantized_features = list()
  if missing_feature:
    for i in range(total_hours):
      quantized_time = admin_time + timedelta(hours=i)
      quantized_value = np.nan #지금은 dic 형태라, pandas 쓰면 NAN이랑 동일, carry forward 도 가능
      quantized_features.append((quantized_time, quantized_value))
  else:
    feature_list = list(sorted(feature_list, key=lambda x: x[0]))
    for i in range(total_hours):
      quantized_time = admin_time + timedelta(hours=i)
      quantized_time_start = quantized_time - timedelta(hours=1)
      quantized_values = list(filter(lambda x: quantized_time_start < x[0] <= quantized_time, feature_list)) #filter함수 True만 필터
      if len(quantized_values) > 0:
        quantized_value = quantized_values[-1][1]
      else:
        quantized_value = np.nan 

      quantized_features.append((quantized_time, quantized_value))
  
  return quantized_features

In [17]:
## 전체 데이터에 대해서 데이터 변환하기
quantized_features = dict()
for hadm_id in tqdm(Extracted_HADMIDS, desc='Doing Quantization'):
    feature_keys = ['PULSE', 'SBP', 'DBP', 'RESP', 'TEMP','FiO2','MAP', 'BILIRUBIN', 'WBC', 'HEMATOCRIT','PH' ,'LACTATE', 'PLATELET', 'CREATININE', 'SODIUM','POTASSIUM']
    admin_time = admin_disc_time[hadm_id]['ADMIN']
    disc_time = admin_disc_time[hadm_id]['DISC']
    if (disc_time - admin_time).total_seconds() / (60*60) < 1.0:
      continue
    quantized_features[hadm_id] = dict()
    for kk in feature_keys:
      if kk in extracted_features[hadm_id]:
        feature_list = extracted_features[hadm_id][kk]
        quantized_features[hadm_id][kk] = quantize_features(feature_list, admin_time, disc_time)
      else:
        quantized_features[hadm_id][kk] = quantize_features(None, admin_time, disc_time, missing_feature=True)


Doing Quantization:   0%|          | 0/32812 [00:00<?, ?it/s]

In [18]:
NORMAL_VALUE = dict({
    'PULSE': 80.0,
    'SBP': 120.0,
    'DBP': 80.0,
    'RESP': 14.0,
    'TEMP': 36.5,
    'BILIRUBIN': 0.6,
    'WBC': 7.0,
    'HEMATOCRIT': 45.0,
    'LACTATE': 0.7,
    'PLATELET': 300.,
    'CREATININE': 0.8,
    'SODIUM': 140.,
    'FiO2' : 400,
    'PH' : 7.35,
    'POTASSIUM' : 4.25,
    'MAP' : 85
})

In [19]:
def carry_forward(feature_name, feature_list):
  imputed_features = list()
  for i, (feature_t, feature_v) in enumerate(feature_list):
    if np.isnan(feature_v):
      last_observations = list(filter(lambda x: (x[0] <= feature_t) and (not np.isnan(x[1])), feature_list))
      if len(last_observations) > 0:
        imputed_features.append((feature_t, last_observations[-1][1]))
      else:
        imputed_features.append((feature_t, NORMAL_VALUE[feature_name]))
    else:
      imputed_features.append((feature_t, feature_v))

  return imputed_features

In [20]:
Quantized_HADMIDS = list(quantized_features.keys())
carry_forwarded_features = dict()
feature_keys = ['PULSE', 'SBP', 'DBP', 'RESP', 'TEMP','FiO2','MAP', 'BILIRUBIN', 'WBC', 'HEMATOCRIT','PH' ,'LACTATE', 'PLATELET', 'CREATININE', 'SODIUM','POTASSIUM']
for hadm_id in tqdm(Quantized_HADMIDS, desc='Carry-Forward'):
  carry_forwarded_features[hadm_id] = dict()

  for kk in feature_keys:
    feature_list = quantized_features[hadm_id][kk]
    carry_forwarded_features[hadm_id][kk] = carry_forward(kk, feature_list)

Carry-Forward:   0%|          | 0/32785 [00:00<?, ?it/s]

In [21]:
#항목별 정상값의 범위
#수정 필요
FEATURE_NORMAL_RANGE = dict({
    'PULSE': (0, 300),
    'SBP': (0, 300),
    'DBP': (0, 300),
    'RESP': (0, 120),
    'TEMP': (25, 50),
    'BILIRUBIN': (0, 75),
    'WBC': (0, 100),
    'HEMATOCRIT': (0, 100),
    'LACTATE': (0, 20),
    'PLATELET': (0, 1000),
    'CREATININE': (0, 20),
    'SODIUM': (0, 500),
    'FiO2' : (300, 500),
    'PH' : (7.35, 7.45),
    'POTASSIUM' : (3.7, 5.2),
    'MAP' : (70, 100)
})

In [52]:
#Min-Max Normalization
min_max_normalized = dict()
for hadm_id in tqdm(Quantized_HADMIDS, desc='Min-Max Normalization', ncols=700):
  min_max_normalized[hadm_id] = dict()
  feature_keys = ['PULSE', 'SBP', 'DBP', 'RESP', 'TEMP','FiO2','MAP', 'BILIRUBIN', 'WBC', 'HEMATOCRIT','PH' ,'LACTATE', 'PLATELET', 'CREATININE', 'SODIUM','POTASSIUM']
  for kk in feature_keys:
    features = np.array(carry_forwarded_features[hadm_id][kk])

    min = FEATURE_NORMAL_RANGE[kk][0]
    max = FEATURE_NORMAL_RANGE[kk][1]

    features[:, 1] = (features[:, 1] - min) / (max - min)
    min_max_normalized[hadm_id][kk] = features

Min-Max Normalization:   0%|                                                                                  …

In [53]:
total_death_included_hadm_ids = list(death_labels.keys())
total_los_included_hadm_ids = list(los_labels.keys())
feature_keys = ['PULSE', 'SBP', 'DBP', 'RESP', 'TEMP','FiO2','MAP', 'BILIRUBIN', 'WBC', 'HEMATOCRIT','PH' ,'LACTATE', 'PLATELET', 'CREATININE', 'SODIUM','POTASSIUM']

In [56]:
## 사망 예측용 데이터
death_dataset = dict({
    'HADM_IDS': list(),
    'LABEL': list(),
    'DATA': list()
})

for hadm_id in tqdm(total_death_included_hadm_ids, desc='Make Death Dataset', ncols=700):
  if hadm_id not in min_max_normalized:
    continue
  if len(min_max_normalized[hadm_id][feature_keys[0]]) >= 24:
    time_len = 24
  else:
    time_len = len(min_max_normalized[hadm_id][feature_keys[0]])
  
  try:
    data = np.array(min_max_normalized[hadm_id][feature_keys[0]])[:time_len, 1].reshape(time_len, 1)
  except:
    continue

  for kk in feature_keys[1:]:
    data = np.concatenate((data, np.array(min_max_normalized[hadm_id][kk])[:time_len, 1].reshape(time_len, 1)), axis=1)
  
  death_dataset['HADM_IDS'].append(hadm_id)
  death_dataset['LABEL'].append(death_labels[hadm_id]['DEATH'])
  death_dataset['DATA'].append(data)

Make Death Dataset:   0%|                                                                                     …

In [57]:
## 재실기간 예측용 데이터
los_dataset = dict({
    'HADM_IDS': list(),
    'LABEL': list(),
    'DATA': list()
})

for hadm_id in tqdm(total_los_included_hadm_ids, desc='Make LOS Dataset', ncols=700):
  time_len = 12
  
  try:
    data = np.array(min_max_normalized[hadm_id][feature_keys[0]])[:time_len, 1].reshape(time_len, 1)
  except:
    continue

  for kk in feature_keys[1:]:
    data = np.concatenate((data, np.array(min_max_normalized[hadm_id][kk])[:time_len, 1].reshape(time_len, 1)), axis=1)
  
  los_dataset['HADM_IDS'].append(hadm_id)
  los_dataset['LABEL'].append(los_labels[hadm_id])
  los_dataset['DATA'].append(data)

Make LOS Dataset:   0%|                                                                                       …

In [101]:
# with open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_dataset_death_HADM_IDS.p', 'wb') as f:
#     pickle.dump(death_dataset['HADM_IDS'], f)
# with open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_dataset_death_LABEL.p', 'wb') as f:
#     pickle.dump(death_dataset['LABEL'], f)
# with open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_dataset_death_DATA.p', 'wb') as f:
#     pickle.dump(death_dataset['DATA'], f)
# death_dataset_read = dict({
#     'HADM_IDS': list(),
#     'LABEL': list(),
#     'DATA': list()
# })
# with open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_dataset_death_HADM_IDS.p', 'rb') as f:
#     death_dataset_read['HADM_IDS'] = pickle.load(f)
# #load
# with open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_dataset_death.pickle', 'rb') as f:
#     death_dataset_read = pickle.load(f)

# # gzip 압축 파일을 열기
# with gzip.open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_dataset_death.pickle.gz', 'rb') as f:
#     death_dataset_read = pickle.load(f)

In [77]:
pd.DataFrame(zip(death_dataset['HADM_IDS'],death_dataset['LABEL'] ,death_dataset['DATA'])).head()

,0,1,2
0,26184834,1,"[[0.26666666666666666, 0.4, 0.2666666666666666..."
1,23581541,0,"[[0.26666666666666666, 0.4, 0.2666666666666666..."
2,20345487,1,"[[0.26666666666666666, 0.4, 0.2666666666666666..."
3,23822395,0,"[[0.26666666666666666, 0.4, 0.2666666666666666..."
4,28994087,0,"[[0.26666666666666666, 0.4, 0.2666666666666666..."
